In [6]:
import numpy as np
from classifiers import *
from data_generation import *
from timeit import default_timer as timer

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Dataset

In [8]:
train_path = 'datasets/mnist_train.csv'
test_path = 'datasets/mnist_test.csv'
y_col = 0

In [9]:
def get_features_and_labels(dataset, y_col):
    y = dataset[:, y_col:y_col+1]
    selector = [col for col in range(dataset.shape[1]) if col != y_col]
    X = dataset[:, selector]
    return X, y

In [10]:
## Import data and get sets
train_set = np.genfromtxt(train_path, delimiter=',')
test_set = np.genfromtxt(test_path, delimiter=',')

train_X, train_y = get_features_and_labels(train_set, y_col)
test_X, test_y = get_features_and_labels(test_set, y_col)

In [11]:
# Optional: Truncate datasets/mess around with them as needed
n = 1000
train_X, train_y = train_X[0:n, :], train_y[0:n, :]
test_X, test_y = test_X[0:n, :], test_y[0:n, :]

#Normalize Data
train_X, train_y = train_X - np.mean(train_X) , train_y
test_X, test_y = test_X - np.mean(test_X) , test_y

In [12]:
n = test_y.size
train_y = train_y.astype(int)
test_y = test_y.astype(int)

# Run Quantile Regression 

In [13]:
def train_classifiers(X_train, y_train, X_test, y_test, a, reg_param=1,
                      kernel='linear', kernel_param=1, loss_function='logistic',
                      opt_type='SGD', opt_params={'learning_rate': 1}):
    results = {}
    
    base = opt_params['plot_file']
    opt_params['plot_file'] = 'IT' + base
    start = timer()
    clf5 = QuantileIT(gamma=a, alpha=reg_param, kernel_type=kernel, opt_type=opt_type, opt_params=opt_params,
                              kernel_param=kernel_param, loss_function=loss_function)
    clf5.fit(X_train, y_train)
    preds_5 = clf5.predict(X_test)
    end = timer()
    
    abs_loss = weighted_absolute_loss(preds_5, y_test, a)
    print('Weighted Absolute Loss of QuantileIT, gamma=' + str(a) + ' %s' %
          abs_loss)
    
    preds_5_in = clf5.predict(X_train)
    abs_loss_in = weighted_absolute_loss(preds_5_in, y_train, a)
    print('In Sample Weighted Absolute Loss of QuantileIT, gamma=' + str(a) + ' %s' %
          abs_loss_in)
    print('Time Elapsed: ' + str(end - start))
    
    results[clf5] = {'AbsLoss': abs_loss, 'Preds': preds_5}
    
    print(' ')

    opt_params['plot_file'] = 'AT' + base
    start = timer()
    clf6 = QuantileAT(gamma=a, alpha=reg_param, kernel_type=kernel, opt_type=opt_type, opt_params=opt_params,
                              kernel_param=kernel_param, loss_function=loss_function)
    clf6.fit(X_train, y_train)
    preds_6 = clf6.predict(X_test)
    end = timer()
    abs_loss = weighted_absolute_loss(preds_6, y_test, a)
    print('Weighted Absolute Loss of QuantileAT, gamma=' + str(a) + ' %s' %
          abs_loss)
    
    preds_6_in = clf6.predict(X_train)
    print('In Sample Weighted Absolute Loss of QuantileAT, gamma=' + str(a) + ' %s' %
          abs_loss_in)
    print('Time Elapsed: ' + str(end - start))
    
    results[clf6] = {'AbsLoss': abs_loss, 'Preds': preds_6}
    return results

In [14]:
reg = 10
kern = 'linear'
kern_param = 1
lf = 'hinge'

parms = {'learning_rate': 0.00000001, 'momentum_gamma': 0.9,
         'batch_size': 500, 'plot_file': 'test.png'}

results = train_classifiers(train_X, train_y, test_X, test_y, 0.3, reg_param=reg,
                            kernel=kern, kernel_param=kern_param, loss_function=lf,
                            opt_type='SGD', opt_params=parms)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Generating plot ITtest.png
Weighted Absolute Loss of QuantileIT, gamma=0.3 0.7327
In Sample Weighted Absolute Loss of QuantileIT, gamma=0.3 0.3685
Time Elapsed: 83.846456031
 
Generating plot ATtest.png
Weighted Absolute Loss of QuantileAT, gamma=0.3 0.7690999999999999
In Sample Weighted Absolute Loss of QuantileAT, gamma=0.3 0.3685
Time Elapsed: 76.90220331199998


# Run Multiclass Test
Learning $3 \lg k$ quantiles

In [142]:
surrogate='IT'
num_quantiles = 7
gammas= [i / (num_quantiles + 1) for i in range(1, num_quantiles + 1)]
alphas=[100]*num_quantiles
kernel_type='linear'
kernel_param = 10
loss_function='hinge'
opt_type = 'Momentum'
opt_params={'learning_rate': 1e-8, 'momentum_gamma': 0.9, 'batch_size': 100}

In [143]:
clf = QuantileMulticlass(surrogate=surrogate, gammas=gammas, alphas=alphas, 
                         kernel_type=kernel_type, loss_function=loss_function, 
                         opt_type=opt_type, opt_params=opt_params)

In [144]:
clf.fit(train_X, train_y)

OSError: [Errno 24] Too many open files

In [ ]:
preds = clf.predict(test_X)

In [ ]:
weighted_absolute_loss(preds, test_y, 0.5)

In [ ]:
metrics.zero_one_loss(preds, test_y)

In [ ]:
np.concatenate((preds,test_y), axis=1)

# Run Multiclass Algorithms (for Comparison)

In [137]:
# Standard Logisitic regression
from sklearn.linear_model import LogisticRegression
clf_log = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', fit_intercept=False).fit(train_X, train_y)
metrics.zero_one_loss(clf_log.predict(test_X), test_y)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.17100000000000004

In [138]:
weighted_absolute_loss(clf_log.predict(test_X), test_y, 0.5)

0.286